# 准备模型COMETS
该类模型用于存储在COMETS模拟中使用的全基因组尺度代谢模型。通常情况下，我们首先会使用COBRAPy加载一个模型。然后，我们可以将这个模型传递给COMETS模型类，这使得我们可以修改COMETS特有的模型参数，比如初始种群大小

In [1]:
import cobra
import cobra.io
import cometspy as c

# Load a textbook example model using the COBRAPy toolbox 
test_model = cobra.io.load_model('textbook')

# Use the above model to create a COMETS model
test_model = c.model(test_model)

# Change comets specific parameters, e.g. the initial biomass of the model
# Notre 
test_model.initial_pop = [0, 0, 1e-7] 

C:\Users\99374\.conda\envs\CarveMe\Lib\site-packages\cometspy\model.py:623: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.smat = pd.concat([self.smat, cdf])
C:\Users\99374\.conda\envs\CarveMe\Lib\site-packages\cometspy\model.py:636: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.


# 设置仿真参数COMETS
COMETS模拟的参数存储在params类中，该类仅包含一个dict对象，其中包含了参数名称及其对应的值。如果我们不带参数初始化此类，它将包含默认的参数值。一旦加载完成后，可以按照需求查看和修改这些参数值。

In [2]:
# Create a parameters object with default values 
my_params = c.params()

# Change the value of a parameter, for example number of simulation cycles
my_params.set_param('maxCycles', 100)

# Set some writeTotalBiomassLog parameter to True, in order to save the output
my_params.set_param('writeTotalBiomassLog', True)

# See avaliable parameters and their values
my_params.show_params()

,VALUE,UNITS
BiomassLogName,biomass,
BiomassLogRate,1,cycles
FluxLogName,flux_out,
FluxLogRate,5,cycles
MediaLogName,media_out,
...,...,...
writeBiomassLog,False,logical
writeFluxLog,False,logical
writeMediaLog,False,logical
writeSpecificMediaLog,False,logical


# 准备模拟布局COMETS
layout类描述了环境，即“word”的特性，包括其中存在的物种（models）。它可以为空实例化或使用COMETS模型进行实例化：
  1.若不带任何参数实例化（例如：my_layout = c.layout()），则会创建一个空布局，并带有所有需要填充的必要字段。
  2.如果通过传入一个模型（或多个模型）来实例化布局，则会生成一个布局，其中包含所有这些模型能够与环境交换的代谢物，其初始浓度为零，以及无限浓度（默认-1000）的金属离子。
为了查看Comets布局的不同部分，让我们首先从上述已加载的教科书模型创建一个布局：

In [3]:
my_layout = c.layout(test_model)

C:\Users\99374\.conda\envs\CarveMe\Lib\site-packages\cometspy\layout.py:1361: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.media = pd.concat([self.media, new_row],


在该环境中，layout类存储了关于物种（my_layout.models）和空间结构（my_layout.grid）的信息。在这个例子中，模型仅指教科书模型，而网格则是默认的单格结构，即只有一个单元格。

此外，layout还以pandas数据帧的形式存储了介质信息。在这种情况下，没有预先设置任何介质成分的量。这意味着初始化时，环境中对于模型可以交换的所有代谢物及其他介质成分，其初始浓度都是零或按照默认情况设定（如金属离子为无限浓度）。

In [4]:
my_layout.media

,diff_c,g_refresh,g_static,g_static_val,init_amount,metabolite
0,0.000005,0,0,0,0,ac_e
1,0.000005,0,0,0,0,acald_e
2,0.000005,0,0,0,0,akg_e
3,0.000005,0,0,0,0,co2_e
4,0.000005,0,0,0,0,etoh_e
5,0.000005,0,0,0,0,for_e
6,0.000005,0,0,0,0,fru_e
7,0.000005,0,0,0,0,fum_e
8,0.000005,0,0,0,0,glc__D_e
9,0.000005,0,0,0,0,gln__L_e


当根据模型初始化时，环境中可能存在的介质化合物是至少在一个模型中有交换反应的所有化合物。上表所示的介质是一个pandas数据帧，其中存储了多种信息：

- init_amount 表示在模拟网格每个单元格中添加的初始量（单位为mmol）。
- diff_c 指示分子是否具有不同于默认扩散常数（存储在ec_layout.global_diff中）的扩散系数。
- g_static 表示该组分是否应保持静态值，即不受模拟消耗和其他效应的影响，例如，用于设定某些营养物质为无限供应。
- g_static_val 表示如果前一列（g_static）值为1，则该组分应保持的静态值是多少。
- g_refresh_val 表示在每次模拟循环后应向网格每个单元格中添加的代谢物量。

此外，我们还可以设置特定于模拟网格中某个单元格的局部静态值和刷新值。

当介质组件被设置为静态时，这意味着其浓度在每个循环后都会恢复到设定的静态值。这在我们希望某个介质组分在模拟过程中保持近乎无限供应时使用。

当一个介质组件有刷新值时，这意味着它将在每个模拟周期通过添加设定的数量得到补充。

局部刷新值存储在一个列表my_layout.local_refresh中，列表中的每个元素本身也是一个列表形式为 [ x  y  m1_r  m2_r  m3_r ... ]，前两个元素x和y代表坐标，后面跟随的是所有代谢物按照媒体中的顺序排列的刷新值。

局部静态值也以类似的方式存储。my_layout.local_static列表中的每个元素本身也是一个列表形式为 [ x  y  m1_s  m1_s_v m2_s  m2_s_v ... ]。这里不同的是，对于每种代谢物，都有两个值，一个定义该代谢物在该坐标处是否应保持静态（m1_s, m2_s, ...），另一个则表示应保持的数值（m1_s_v, m2_s_v, ...）。

请注意，local_refresh 和 local_static 可能都为空（默认情况），或者仅包含对应坐标存在至少一个非零刷新值或静态值的条目。

最后，layout还包含了每个模型起始生物量的信息。这些信息存储在initial_pop列表中。initial_pop的每个组成部分本身就是一个格式为[x y biomass_1 biomass_2 ...]的列表，用于指定每个坐标处每个模型的生物量数量。

# 运行COMETS模拟 
comets类利用layout对象和parameters对象来运行模拟并存储输出结果。运行一个COMETS模拟相当直接。我们首先通过传递layout对象和params对象作为参数来定义comets对象，然后调用run()方法执行模拟：

In [5]:
my_simulation = c.comets(my_layout, my_params)
my_simulation.run()


Running COMETS simulation ...
Done!


# 检查模拟输出及可能的错误
该命令在后台启动COMETS Java引擎，在控制台中执行并生成标准输出（stdout）及标准错误（stderr）日志记录。这些日志可通过各自对应的字段“run_outputs”和“run_errors”进行访问查阅。

In [6]:
print(my_simulation.run_output)

-script
running script file: D:\HZAU\bacteria\cometes\cometespy/.current_script_0x1565e898990
Current Java version: 22.0.1
Parameters file .current_global_0x1565e898990 loaded.
Parameters file .current_package_0x1565e898990 loaded.
Loading layout file '.current_layout_0x1565e898990'...
null\COMETS_manifest.txt (系统找不到指定的路径。)
Unable to initialize manifest file. 
Continuing without writing manifest file.
Found 1 model files!
Loading '.\e_coli_core.cmd' ...
Done!
 Testing default parameters...
Done!
Optimizer status code = 5 (looks ok!)
objective solution = 0.8739215069684305
Constructing world...
Done!
medialist	ac_e	acald_e	akg_e	co2_e	etoh_e	for_e	fru_e	fum_e	glc__D_e	gln__L_e	glu__L_e	h2o_e	h_e	lac__D_e	mal__L_e	nh4_e	o2_e	pi_e	pyr_e	succ_e
Cycle 1
Total biomass:
Model e_coli_core.cmd: 1.0E-7
Cycle 2
Total biomass:
Model e_coli_core.cmd: 1.0E-7
Cycle 3
Total biomass:
Model e_coli_core.cmd: 1.0E-7
Cycle 4
Total biomass:
Model e_coli_core.cmd: 1.0E-7
Cycle 5
Total biomass:
Model e_coli_c

In [7]:
print(my_simulation.run_errors)

STDERR empty.


# 访问模拟结果
成功模拟的结果存储在comets对象内的多个字段中，这取决于参数writeTotalBiomasslog、writeBiomassLog、writeFluxLog和writeMediaLog是否设置为True。

total_biomass字段存储了每个时间点和物种的总生物量（汇总所有坐标上的生物量）。
biomass字段详细记录了每个时间点、坐标以及物种的具体生物量值。
media字段存储了各个时间点培养基的组成情况。
fluxes字段存储了每个物种、坐标和时间点的代谢通量数据。
此外，特定的COMETS模式会有额外的输出字段；例如，如果我们运行的是进化模拟，genotypes字段会存储每个物种的相关信息，如其祖先以及它所经历的突变等。

所有输出文件都是pandas DataFrame格式，可以使用标准Python工具进一步分析或绘制图表。



In [8]:
my_simulation.total_biomass

,cycle,e_coli_core
0,0,1.000000e-07
1,1,1.000000e-07
2,2,1.000000e-07
3,3,1.000000e-07
4,4,1.000000e-07
...,...,...
96,96,1.000000e-07
97,97,1.000000e-07
98,98,1.000000e-07
99,99,1.000000e-07
